In [7]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.autograd.variable import Variable
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.preprocessing import MinMaxScaler
import time
from sklearn.metrics import mean_squared_error

lookback = 100 # choose sequence length
input_dim = 1
hidden_dim = 128
num_layers = 2
output_dim = 1
num_epochs = 100


device = 'cuda' if torch.cuda.is_available()  else 'cpu'


def split_data(stock, lookback):
  #  data_raw = stock.to_numpy() # convert to numpy array Do not need this if you run minmaxscaler
    data_raw = stock
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - lookback): 
        data.append(data_raw[index: index + lookback])
    
    data = np.array(data);
    test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);
    
    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]
    
    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]
    
    return [x_train, y_train, x_test, y_test]


class MyDataset(Dataset):
    def __init__(self, data, pred, transform=None):
        self.data = data.float()
        self.pred = pred.float()
        self.transform = transform

    def __getitem__(self, index):
        x = self.data[index]
        y = self.pred[index]
        if self.transform:
            x = self.transform(x)
            y = self.transform(y)
        return x, y

    def __len__(self):
        return len(self.data)
    

class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out
    

    



data = pd.read_csv("data.csv")[-100000:]
data.head()
data.max(axis=0)
data.min(axis=0)
minmax = MinMaxScaler()

# Picking a day to predict the stock prices for the 10 mins of the next day
price = data[['5']]
price = minmax.fit_transform(price)
x_train, y_train, x_test, y_test = split_data(price, lookback)


x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train= torch.from_numpy(y_train).type(torch.Tensor)
y_test = torch.from_numpy(y_test).type(torch.Tensor)


dataset = MyDataset(x_train, y_train)
loader = DataLoader(dataset, batch_size = 32, shuffle = False)


model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers).to(device)
criterion = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


hist = np.zeros(num_epochs)
start_time = time.time()
error = []

for t in range(num_epochs):
    #Minibatching for Stochastic Gradient Descent 
    count = 0
    for n_batch, batch in enumerate(loader):
#         if count%1000 == 0:
#             print("Epoch ", t, "MSE: ", loss.item())
        n_data = Variable(batch[0].to(device), requires_grad=True)
        ground_truth = Variable(batch[1].to(device), requires_grad=True)
        
        optimizer.zero_grad()
        y_train_pred = model(n_data)
        

        loss = criterion(y_train_pred, ground_truth)
        loss.backward()
        optimizer.step()
        
    error.append(loss)
    hist[t] = loss.item()
    


#### compute error and create output file
# creating output file
prediction = torch.reshape(prediction, [1,len(prediction)]).tolist()[0]
truth = [elem[0] for elem in y_train.tolist()]

rmse = mean_squared_error(truth,prediction)

df = pd.DataFrame({'Real Price': truth,
                   'Predicted Price': prediction,
                   'RMS Error': [rmse]+['']*(len(truth)-1)})
df.to_csv('out_1.csv',index=False)

Epoch  0 MSE:  0.00022330868523567915
Epoch  0 MSE:  0.0035905318800359964
Epoch  0 MSE:  0.00014922759146429598
Epoch  0 MSE:  0.0012071639066562057
Epoch  0 MSE:  0.0016400570748373866
Epoch  0 MSE:  0.001280297408811748
Epoch  0 MSE:  0.0006998958415351808
Epoch  0 MSE:  0.0019314405508339405
Epoch  0 MSE:  0.0038344217464327812
Epoch  0 MSE:  0.005944296717643738
Epoch  0 MSE:  0.004619073122739792
Epoch  0 MSE:  0.004500702954828739
Epoch  0 MSE:  0.0015281648375093937
Epoch  0 MSE:  0.0008198514697141945
Epoch  0 MSE:  0.0001858137984527275
Epoch  0 MSE:  0.00023530033649876714
Epoch  0 MSE:  0.002609441988170147
Epoch  0 MSE:  0.0035117182414978743
Epoch  0 MSE:  0.002405376173555851
Epoch  0 MSE:  0.001525976462289691
Epoch  0 MSE:  0.0004967910936102271
Epoch  0 MSE:  6.970603135414422e-05
Epoch  0 MSE:  0.0003771319461520761
Epoch  0 MSE:  0.0035606238525360823
Epoch  0 MSE:  0.0021416994277387857
Epoch  0 MSE:  0.0020279069431126118
Epoch  0 MSE:  0.0009775182697921991
Epoch

Epoch  0 MSE:  3.661835580714978e-05
Epoch  0 MSE:  0.00011355148308211938
Epoch  0 MSE:  0.00022600559168495238
Epoch  0 MSE:  0.00032356148585677147
Epoch  0 MSE:  0.0005562748992815614
Epoch  0 MSE:  0.0009763396810740232
Epoch  0 MSE:  0.0014168365160003304
Epoch  0 MSE:  0.0010153031907975674
Epoch  0 MSE:  0.0023826793767511845
Epoch  0 MSE:  0.002164307050406933
Epoch  0 MSE:  0.0012144140200689435
Epoch  0 MSE:  0.0006923716282472014
Epoch  0 MSE:  7.793690747348592e-05
Epoch  0 MSE:  0.00048093084478750825
Epoch  0 MSE:  0.0005195423145778477
Epoch  0 MSE:  0.0008427116554230452
Epoch  0 MSE:  0.0021632174029946327
Epoch  0 MSE:  0.002613321878015995
Epoch  0 MSE:  0.00257498724386096
Epoch  0 MSE:  0.0012320687528699636
Epoch  0 MSE:  0.00011009182344423607
Epoch  0 MSE:  2.256719562865328e-05
Epoch  0 MSE:  0.00024285427934955806
Epoch  0 MSE:  0.0010056535247713327
Epoch  0 MSE:  0.0013308200286701322
Epoch  0 MSE:  0.0016456199809908867
Epoch  0 MSE:  0.0011919725220650434

Epoch  0 MSE:  0.00028405460761860013
Epoch  0 MSE:  0.0001537149801151827
Epoch  0 MSE:  0.0006557613960467279
Epoch  0 MSE:  0.0005296687595546246
Epoch  0 MSE:  0.00038552351179532707
Epoch  0 MSE:  0.0006524129421450198
Epoch  0 MSE:  0.0008570456993766129
Epoch  0 MSE:  0.0013593747280538082
Epoch  0 MSE:  0.0016433888813480735
Epoch  0 MSE:  0.001196281868033111
Epoch  0 MSE:  0.0003123976057395339
Epoch  0 MSE:  0.0003174587618559599
Epoch  0 MSE:  4.789734157384373e-05
Epoch  0 MSE:  0.00012598390458151698
Epoch  0 MSE:  2.8603522878256626e-05
Epoch  0 MSE:  2.1354244381655008e-05
Epoch  0 MSE:  1.7333457435597666e-05
Epoch  0 MSE:  8.002914910321124e-06
Epoch  0 MSE:  4.3056526919826865e-05
Epoch  0 MSE:  0.00032456422923132777
Epoch  0 MSE:  0.0005288936663419008
Epoch  0 MSE:  0.0006063367472961545
Epoch  0 MSE:  0.00041763728950172663
Epoch  0 MSE:  0.0003516862343531102
Epoch  0 MSE:  0.0002353121672058478
Epoch  0 MSE:  0.00014945233124308288
Epoch  0 MSE:  0.000105296974

Epoch  0 MSE:  0.0021551535464823246
Epoch  0 MSE:  0.0018295632908120751
Epoch  0 MSE:  0.001686723786406219
Epoch  0 MSE:  0.0007999785011634231
Epoch  0 MSE:  0.0004382553161121905
Epoch  0 MSE:  0.0019889073446393013
Epoch  0 MSE:  0.001615771441720426
Epoch  0 MSE:  0.0006041686283424497
Epoch  0 MSE:  0.0014921162510290742
Epoch  0 MSE:  0.006156235933303833
Epoch  0 MSE:  0.0035515690688043833
Epoch  0 MSE:  0.002815828425809741
Epoch  0 MSE:  0.0008196235867217183
Epoch  0 MSE:  8.096524106804281e-05
Epoch  0 MSE:  0.0006748847663402557
Epoch  0 MSE:  0.0010576709173619747
Epoch  0 MSE:  0.0015835028607398272
Epoch  0 MSE:  0.002729671308770776
Epoch  0 MSE:  0.0025875968858599663
Epoch  0 MSE:  0.003923583775758743
Epoch  0 MSE:  0.005643381271511316
Epoch  0 MSE:  0.003069948172196746
Epoch  0 MSE:  0.00219991197809577
Epoch  0 MSE:  0.0016749154310673475
Epoch  0 MSE:  0.0011471007019281387
Epoch  0 MSE:  0.0006455120164901018
Epoch  0 MSE:  0.0005063770222477615
Epoch  0 MS

Epoch  0 MSE:  6.325331924017519e-05
Epoch  0 MSE:  8.464237180305645e-05
Epoch  0 MSE:  0.00013558290083892643
Epoch  0 MSE:  2.8646285500144586e-05
Epoch  0 MSE:  1.2380989574012347e-05
Epoch  0 MSE:  8.383468230022117e-05
Epoch  0 MSE:  6.395302625605837e-05
Epoch  0 MSE:  1.603040800546296e-05
Epoch  0 MSE:  9.412787039764225e-05
Epoch  0 MSE:  0.0004590642638504505
Epoch  0 MSE:  0.00044117728248238564
Epoch  0 MSE:  0.0002131430373992771
Epoch  0 MSE:  0.00023036313359625638
Epoch  0 MSE:  3.7963873182889074e-05
Epoch  0 MSE:  0.00010258239490212873
Epoch  0 MSE:  0.0009430839563719928
Epoch  0 MSE:  0.0022196380887180567
Epoch  0 MSE:  0.002428700216114521
Epoch  0 MSE:  0.0026195666287094355
Epoch  0 MSE:  0.003819580189883709
Epoch  0 MSE:  0.004255744628608227
Epoch  0 MSE:  0.00484006293118
Epoch  0 MSE:  0.0035375896841287613
Epoch  0 MSE:  0.002682278398424387
Epoch  0 MSE:  0.0023359020706266165
Epoch  0 MSE:  0.001612412859685719
Epoch  0 MSE:  0.0014762921491637826
Epoc

...

Epoch  3 MSE:  0.0022280169650912285
Epoch  3 MSE:  0.0011679510353133082
Epoch  3 MSE:  0.0009556938894093037
Epoch  3 MSE:  0.0007599318632856011
Epoch  3 MSE:  0.0007570944144390523
Epoch  3 MSE:  0.0008537024259567261
Epoch  3 MSE:  0.0008699935278855264
Epoch  3 MSE:  0.0014454552438110113
Epoch  3 MSE:  0.0017871353775262833
Epoch  3 MSE:  0.0015707082347944379
Epoch  3 MSE:  0.0012889516074210405
Epoch  3 MSE:  0.0006308467709459364
Epoch  3 MSE:  0.0010634809732437134
Epoch  3 MSE:  0.0011030113091692328
Epoch  3 MSE:  0.0015785526484251022
Epoch  3 MSE:  0.0019844970665872097
Epoch  3 MSE:  0.0012492025271058083
Epoch  3 MSE:  0.001006764592602849
Epoch  3 MSE:  0.0009166624513454735
Epoch  3 MSE:  0.0008981154533103108
Epoch  3 MSE:  0.000797827320639044
Epoch  3 MSE:  0.0006266541313380003
Epoch  3 MSE:  0.00034604404936544597
Epoch  3 MSE:  0.00021599182218778878
Epoch  3 MSE:  0.00022094135056249797
Epoch  3 MSE:  0.0002912594354711473
Epoch  3 MSE:  0.000363945378921926
E

Epoch  3 MSE:  6.807570753153414e-05
Epoch  3 MSE:  0.00013504402886610478
Epoch  3 MSE:  6.629344716202468e-05
Epoch  3 MSE:  5.1259874453535303e-05
Epoch  3 MSE:  0.0003205338725820184
Epoch  3 MSE:  0.00011819772043963894
Epoch  3 MSE:  0.0002387103741057217
Epoch  3 MSE:  0.00027620719629339874
Epoch  3 MSE:  0.00017205743642989546
Epoch  3 MSE:  0.00014314588042907417
Epoch  3 MSE:  0.0003268908476457
Epoch  3 MSE:  0.00016092171426862478
Epoch  3 MSE:  0.0001573538756929338
Epoch  3 MSE:  0.00015792390331625938
Epoch  3 MSE:  0.00014217736315913498
Epoch  3 MSE:  0.0012626665411517024
Epoch  3 MSE:  0.0021819390822201967
Epoch  3 MSE:  0.0019910181872546673
Epoch  3 MSE:  0.0020575353410094976
Epoch  3 MSE:  0.0024321959353983402
Epoch  3 MSE:  0.0029135197401046753
Epoch  3 MSE:  0.0018734496552497149
Epoch  3 MSE:  0.0004280239518266171
Epoch  3 MSE:  0.00047837913734838367
Epoch  3 MSE:  0.00027062444132752717
Epoch  3 MSE:  0.00033306758268736303
Epoch  3 MSE:  0.000648328743

Epoch  3 MSE:  0.0003728153824340552
Epoch  3 MSE:  0.00034829246578738093
Epoch  3 MSE:  0.00024545445921830833
Epoch  3 MSE:  0.00025221757823601365
Epoch  3 MSE:  0.00027917552506551147
Epoch  3 MSE:  0.0007689787307754159
Epoch  3 MSE:  0.0007176629733294249
Epoch  3 MSE:  0.0006182565120980144
Epoch  3 MSE:  0.00047661454300396144
Epoch  3 MSE:  0.00024272948212455958
Epoch  3 MSE:  0.00023290340323001146
Epoch  3 MSE:  0.00019959146447945386
Epoch  3 MSE:  4.37704220530577e-05
Epoch  3 MSE:  6.215299072209746e-05
Epoch  3 MSE:  8.651884854771197e-05
Epoch  3 MSE:  0.0001574744237586856
Epoch  3 MSE:  0.00031992775620892644
Epoch  3 MSE:  0.001058640074916184
Epoch  3 MSE:  0.0021808850578963757
Epoch  3 MSE:  0.0013065835228189826
Epoch  3 MSE:  0.00048709235852584243
Epoch  3 MSE:  0.00015539475134573877
Epoch  3 MSE:  1.588927443663124e-05
Epoch  3 MSE:  4.7839370381552726e-05
Epoch  3 MSE:  0.0008440183009952307
Epoch  3 MSE:  0.0021114982664585114
Epoch  3 MSE:  0.00207187095

Epoch  3 MSE:  0.010049120523035526
Epoch  3 MSE:  0.008972639217972755
Epoch  3 MSE:  0.00787277054041624
Epoch  3 MSE:  0.007110311184078455
Epoch  3 MSE:  0.005188025068491697
Epoch  3 MSE:  0.004726034589111805
Epoch  3 MSE:  0.003734757425263524
Epoch  3 MSE:  0.0031303877476602793
Epoch  3 MSE:  0.001992896432057023
Epoch  3 MSE:  0.002509261714294553
Epoch  3 MSE:  0.0018985208589583635
Epoch  3 MSE:  0.001813175156712532
Epoch  3 MSE:  0.0016160744708031416
Epoch  3 MSE:  0.001469458220526576
Epoch  3 MSE:  0.0013690126361325383
Epoch  3 MSE:  0.0012085726484656334
Epoch  3 MSE:  0.0011295047588646412
Epoch  3 MSE:  0.0011030053719878197
Epoch  3 MSE:  0.0012706127017736435
Epoch  3 MSE:  0.0023271082900464535
Epoch  3 MSE:  0.0019350486109033227
Epoch  3 MSE:  0.0012745439307764173
Epoch  3 MSE:  0.0016578378854319453
Epoch  3 MSE:  0.0025319568812847137
Epoch  3 MSE:  0.0031214472837746143
Epoch  3 MSE:  0.003929045516997576
Epoch  3 MSE:  0.004014914855360985
Epoch  3 MSE:  

KeyboardInterrupt: 